In [1]:
%config IPCompleter.greedy=True

In [2]:
import os
#os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:2.4.3 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS']

'--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.3 pyspark-shell'

In [3]:
from pyspark import SparkContext
import json


In [4]:
sc = SparkContext()
sc.setLogLevel("WARN")

In [5]:
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [6]:
window = 60 # seconds
ssc = StreamingContext(sc, window)

In [7]:
kafkaStream = KafkaUtils.createStream(
    ssc, 'localhost:2181', 'spark-streaming', {"clicks":1})

In [8]:
parsed = kafkaStream.map(lambda v: json.loads(v[1]))

In [9]:
parsed.count().map(lambda x:'clicks in this batch: %s' % x).pprint()

In [10]:
my_sorted_data = parsed.transform(lambda rdd: rdd.sortBy(lambda a: a['ts_ingest']))

In [11]:
my_sorted_data.pprint()

In [12]:
ssc.start()
ssc.awaitTermination()

-------------------------------------------
Time: 2019-06-04 13:33:00
-------------------------------------------
clicks in this batch: 15707

-------------------------------------------
Time: 2019-06-04 13:33:00
-------------------------------------------
{'ts_ingest': 1515801793197, 'article_title': 'List of closed railway stations in Britain: P-R', 'visitor_os': 'windows', 'visitor_country': 'BE', 'visitor_page_timer': 14683, 'visitor_browser': 'chrome', 'article': 'https://en.wikipedia.org/wiki/List_of_closed_railway_stations_in_Britain:_P-R', 'visitor_platform': 'desktop', 'visitor_page_height': 21048}
{'ts_ingest': 1515801794407, 'article_title': 'Werauhia marnier-lapostollei', 'visitor_os': 'android', 'visitor_country': 'BE', 'visitor_page_timer': 5847, 'visitor_browser': 'chrome', 'article': 'https://en.wikipedia.org/wiki/Werauhia_marnier-lapostollei', 'visitor_platform': 'mobile', 'visitor_page_height': 4034}
{'ts_ingest': 1515801794411, 'article_title': 'Chaufour-lès-Bonnière

KeyboardInterrupt: 

In [13]:
ssc.stop()